<a href="https://colab.research.google.com/github/ocoropuj/PHYS434/blob/main/PHYS_434_HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frequentist vs Bayesian significance

For a simple counting experiment, the expected background event is $b$ and the observed event is $n$.
The best estimator for signal event $s$ is:
$$s=n-b.$$ In this exercise, we will implement Frequentist significance calculation.

$$ p-value =  \int_{q_{0,n}}^{\infty} f(q_0|b) dq_0 $$
$$Z_{0, Frequentist} = \mathrm{Z score\ 1-tailed\ (p-value)} $$
, where $f(q_0|b)$ is distribution of test statistics $q_0$ in background only hypothesis.
The test statistics is defined as likelihood ratio between backogrund only model and the best fit model
$$ q_0 = -2 \mathrm{ln} \frac{L(n|s=0,b)}{L(n|s,b)} $$
In the simple counting experiment, the Likelihood $L$ is Poisson distribution. The test statistics can be written as
$$ q_0 = -2 \mathrm{ln} \frac{\mathrm{Poisson}(n|b)}{\mathrm{Poisson}(n|s+b)}$$




In [1]:
!pip install iminuit

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import iminuit.minimize as minimize
from tqdm import tqdm
import scipy
from scipy.stats import poisson

# Define test statistics q_0 for Frequentist approach
# 1. We require signal event s >0 for positive signal yield.
#    Therefore, the test statistics q_0 is 0 if N_obs <= Nb
# 2. Compute two Poisson loglikelihood of
#    a) backgorund only model
#    b) signal+background model
#    Evaluate -2 log likelihood ratio between a) and b)

def q0(N_obs, N_b):
    if N_obs <= N_b:
        # If N_obs is less than or equal to N_b, return q0 = 0
        return 0
    else:
        # Calculate the Poisson log likelihood for the background-only model
        log_likelihood_bg = poisson.logpmf(N_obs, N_b)

        # Calculate the Poisson log likelihood for the signal+background model
        log_likelihood_sig_bg = poisson.logpmf(N_obs, N_obs)

        # Calculate -2 * log likelihood ratio
        q0_out = -2 * (log_likelihood_bg - log_likelihood_sig_bg)
        return q0_out

In [3]:
def FreqnetistZ0(N_obs, N_b, n_experiments = 10000):
    # Set a random seed for reproducibility
    np.random.seed(seed=8)

    # Number of toy experiments to generate

    # Step 1: Generate toy experiments and compute q_0 for each experiment
    q_0_values = []

    for _ in range(n_experiments):
        # Generate a random toy experiment based on the background-only model
        toy_experiment = np.random.poisson(N_b)

        # Calculate the q_0 value for this toy experiment
        q_0_value = q0(toy_experiment, N_b)
        q_0_values.append(q_0_value)

    # Step 2: Compute the p-value
    q_0_obs = q0(N_obs, N_b)
    p_value = np.mean(np.array(q_0_values) >= q_0_obs)

    # Step 3: Calculate the Z-score
    Zscore = np.sqrt(2) * scipy.stats.norm.ppf(1 - p_value)

    return Zscore

Now, let's apply our code for numerical calculations.

Consider the case that backogrund only model with yields b=0.5 and observed events n=5.

Calclate discovery significance.

In [4]:
Nobs=5
Nb=0.5

print("Z0freq:%.2f"%(FreqnetistZ0(Nobs,Nb)))

Z0freq:5.26


Q1: How is the number compared to the Baysian signficance from homework5?

We recall that the: $$Z_{0,bayesian}\approx 4.19$$
So we find that: $$Z_{0,frequentist}>Z_{0,bayesian}$$

Consdier a background only model with yields b=4 and observed events n=5. Calculate Baysian signifiance and Frequqnt signficance, and compare the results.

In [5]:
#We recall:
def BayesianZ0(N_obs,N_b):
    # Write your code here
    pvalue = 1-poisson.cdf(N_obs, N_b)
    Zscore= scipy.stats.norm.ppf(1-pvalue)
    return Zscore

In [6]:
Nobs = 5
Nb = 4
print(f'Bayesian: {round(BayesianZ0(Nobs,Nb),4)}')
print(f'Freuentist: {round(FreqnetistZ0(Nobs,Nb),4)}')

Bayesian: 0.7896
Freuentist: 0.4802


We find that the Bayessian significance here is larger, with:
$$Z_{0,frequentist}\approx0.4802; Z_{0,bayesian}\approx0.7896$$
Such that:
$$Z_{0,frequentist}<Z_{0,bayesian}$$
$$$$
This is consistent from the fact that the significance it is calculated two diferent ways, and it could have diferent values for diferent paramenters and observations.